<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/untet_working_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 49.9 MB/s eta 0:00:00


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

train test split

In [4]:
# image_paths_2018 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2018/*.tif")
# image_paths_2019 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2019/*.tif")
# image_paths_2020 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2020/*.tif")
# dem_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/dem/*.tif")
# nwi_ccap_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/nwi_ccap_filled/*.tif")
# label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/ccap_filled/*.tif")

implementing the training set, validation set, and test set split to begin

In [38]:
# this is pulling allllll the data in images and in labels. there's no split among them. need to do that now so I can have some in the validation.

# train_image_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/images/*.tif")
# train_label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/labels/*.tif")


# image_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/images/*.tif")
# label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/labels/*.tif")

image_paths = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/images/*.tif"))
label_paths = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/labels/*.tif"))


# val_image_paths = glob.glob(f"{base_dir}/validation/image/*.tif")
# val_label_paths = glob.glob(f"{base_dir}/validation/label/*.tif")

In [ ]:
image_paths

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [35]:
# def extract_id(file_path):
#     # This function extracts the numeric identifier from a file path, assuming the format you provided
#     return int(os.path.basename(file_path).split('_')[0])

In [42]:
def extract_id(file_path):
    return os.path.basename(file_path).split('.')[0]

identifiers = [extract_id(path) for path in image_paths]  # Assuming image_paths & label_paths are aligned

# Split identifiers into sets for training (70%), validation (15%), and testing (15%)
train_ids, temp_ids = train_test_split(identifiers, test_size=0.3, random_state=42)
validation_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)


In [ ]:
# identifiers
# # Create a DataFrame
# df = pd.DataFrame(identifiers)

# # Print the DataFrame
# print(df)

# # Export path
# export_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/identifiers_001.csv"

# # Export the DataFrame to a CSV file
# df.to_csv(export_path, index=False)

In [45]:
def split_paths(paths, ids_set):
    return [path for path in paths if extract_id(path) in ids_set]

# Split image and label paths according to the identifiers
train_image_paths = split_paths(image_paths, set(train_ids))
validation_image_paths = split_paths(image_paths, set(validation_ids))
test_image_paths = split_paths(image_paths, set(test_ids))

train_label_paths = split_paths(label_paths, set(train_ids))
validation_label_paths = split_paths(label_paths, set(validation_ids))
test_label_paths = split_paths(label_paths, set(test_ids))


In [47]:
import glob
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Define your image and label paths
image_paths = glob.glob("/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/images/*.tif")
label_paths = glob.glob("/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/Data_renamed/labels/*.tif")

# Assuming that every image has a corresponding label with the same name in their respective directories
# Extract base names without extension to pair them
base_names = [os.path.basename(path).split('.')[0] for path in image_paths]

# Split into training, validation, and test sets (70%, 15%, 15%)
train_names, temp_names = train_test_split(base_names, test_size=0.3, random_state=42)
validation_names, test_names = train_test_split(temp_names, test_size=0.5, random_state=42)

# Function to filter paths by identifiers
def filter_paths(paths, ids_set):
    return [path for path in paths if os.path.basename(path).split('.')[0] in ids_set]

# Assigning paths to respective splits based on the split identifiers
train_image_paths = filter_paths(image_paths, set(train_names))
validation_image_paths = filter_paths(image_paths, set(validation_names))
test_image_paths = filter_paths(image_paths, set(test_names))

train_label_paths = filter_paths(label_paths, set(train_names))
validation_label_paths = filter_paths(label_paths, set(validation_names))
test_label_paths = filter_paths(label_paths, set(test_names))

# Create a DataFrame to visualize the distribution
data = {
    "Split": ["Training"] * len(train_image_paths) + ["Validation"] * len(validation_image_paths) + ["Test"] * len(test_image_paths),
    "Image_Path": train_image_paths + validation_image_paths + test_image_paths,
    "Label_Path": train_label_paths + validation_label_paths + test_label_paths
}

df = pd.DataFrame(data)

# Show the DataFrame
print(df)

# If you want to save this table to a CSV file
df.to_csv("/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/split_distribution.csv", index=False)


         Split                                         Image_Path  \
0     Training  /content/drive/MyDrive/DeepLearning_Project/tr...   
1     Training  /content/drive/MyDrive/DeepLearning_Project/tr...   
2     Training  /content/drive/MyDrive/DeepLearning_Project/tr...   
3     Training  /content/drive/MyDrive/DeepLearning_Project/tr...   
4     Training  /content/drive/MyDrive/DeepLearning_Project/tr...   
...        ...                                                ...   
2599      Test  /content/drive/MyDrive/DeepLearning_Project/tr...   
2600      Test  /content/drive/MyDrive/DeepLearning_Project/tr...   
2601      Test  /content/drive/MyDrive/DeepLearning_Project/tr...   
2602      Test  /content/drive/MyDrive/DeepLearning_Project/tr...   
2603      Test  /content/drive/MyDrive/DeepLearning_Project/tr...   

                                             Label_Path  
0     /content/drive/MyDrive/DeepLearning_Project/tr...  
1     /content/drive/MyDrive/DeepLearning_Project/tr...

In [23]:
def extract_id(file_path):
    # This function extracts the numeric identifier from a file path, assuming the format you provided
    # return int(os.path.basename(file_path).split('_')[0])
    return os.path.basename(file_path).split('.')[0]

identifiers = [extract_id(path) for path in image_paths]

# Split identifiers into training, validation, and test sets
train_ids, temp_ids = train_test_split(identifiers, test_size=0.3, random_state=42)
validation_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

# Function to split paths based on their identifiers
def split_paths(paths, ids_set):
    return [path for path in paths if extract_id(path) in ids_set]

# Now, split all your datasets according to these ids
train_image_paths = split_paths(image_paths, set(train_ids))
validation_image_paths = split_paths(image_paths, set(validation_ids))
test_image_paths = split_paths(image_paths, set(test_ids))

In [20]:
# prompt: extract_id function that extracts the information from the file to include information from before the second underscore so itll be like "4697_5847"

# def extract_id(file_path):
#     # This function extracts the numeric identifier from a file path, assuming the format you provided
#     # return int(os.path.basename(file_path).split('_')[0])
#     return os.path.basename(file_path).split('_')[0] + "_" + os.path.basename(file_path).split('_')[1] + "_" + os.path.basename(file_path).split('_')[2]


In [24]:
# Create a unified list of identifiers from your labeled dataset, assuming nwi_ccap_path holds all possible labels
identifiers = [extract_id(path) for path in image_paths]

In [26]:
# Split identifiers into training, validation, and test sets
train_ids, temp_ids = train_test_split(identifiers, test_size=0.3, random_state=42)
validation_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

In [27]:
# Function to split paths based on their identifiers
def split_paths(paths, ids_set):
    return [path for path in paths if extract_id(path) in ids_set]

In [28]:
# Now, split all your datasets according to these ids
train_image_paths = split_paths(image_paths, set(train_ids))
validation_image_paths = split_paths(image_paths, set(validation_ids))
test_image_paths = split_paths(image_paths, set(test_ids))

In [ ]:
test_image_paths

In [ ]:
validation_image_paths

In [ ]:
train_image_paths

In [37]:
# Function to list files in a folder and extract their numeric part
def list_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]
    return files

# List and extract numeric parts from filenames
# images = list_files_and_extract(images_folder)
# labels = list_files_and_extract(labels_folder)

# Find common identifiers
# common_identifiers = set(images.keys()) & set(labels.keys())

# Prepare data for DataFrame
data = []
for identifier in identifiers:
    # Convert identifier to integer
    identifier_int = int(identifier)
    data.append({
        'Identifier': identifier,
        # 'Train_Image_File': train_image_paths[identifier_int],
        'Train_Image_File': train_image_paths[identifier_int % len(train_image_paths)],
        # 'Validation_Image_File': validation_image_paths[identifier_int],
        'Validation_Image_File': validation_image_paths[identifier_int % len(validation_image_paths)],
        # 'Test_Image_File': test_image_paths[identifier_int]
        'Test_Image_File': test_image_paths[identifier_int % len(test_image_paths)]
    })

# Create a DataFrame
df = pd.DataFrame(data)

# Sort the DataFrame by Identifier
df.sort_values(by='Identifier', inplace=True)
df.reset_index(drop=True, inplace=True)

# Add a numeric order column
df.insert(0, 'Number', range(1, len(df) + 1))

# Print the DataFrame
print(df)

# Export path
export_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/train_val_test_001.csv"

# Export the DataFrame to a CSV file
df.to_csv(export_path, index=False)

      Number     Identifier  \
0          1  1000_716_2331   
1          2  1001_716_2331   
2          3  1002_717_2331   
3          4  1003_717_2331   
4          5  1004_717_2331   
...      ...            ...   
2599    2600   997_715_2331   
2600    2601   998_715_2331   
2601    2602   999_715_2331   
2602    2603    99_592_2384   
2603    2604     9_467_2448   

                                       Train_Image_File  \
0     /content/drive/MyDrive/DeepLearning_Project/tr...   
1     /content/drive/MyDrive/DeepLearning_Project/tr...   
2     /content/drive/MyDrive/DeepLearning_Project/tr...   
3     /content/drive/MyDrive/DeepLearning_Project/tr...   
4     /content/drive/MyDrive/DeepLearning_Project/tr...   
...                                                 ...   
2599  /content/drive/MyDrive/DeepLearning_Project/tr...   
2600  /content/drive/MyDrive/DeepLearning_Project/tr...   
2601  /content/drive/MyDrive/DeepLearning_Project/tr...   
2602  /content/drive/MyDrive/DeepLear

In [ ]:
'Train_Image_File': train_image_paths[identifier_int % len(train_image_paths)],

In [ ]:
# Paths to the folders
# images_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images"
# labels_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels"

# Function to list files in a folder and extract their numeric part
def list_files_and_extract(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]
    file_parts = {re.search(r"(\d+_\d+)", f).group(0): f for f in files if re.search(r"(\d+_\d+)", f)}
    return file_parts

# List and extract numeric parts from filenames
# images = list_files_and_extract(images_folder)
# labels = list_files_and_extract(labels_folder)

# Find common identifiers
common_identifiers = set(images.keys()) & set(labels.keys())

# Prepare data for DataFrame
data = []
for identifier in common_identifiers:
    data.append({
        'Identifier': identifier,
        'Image_File': images[identifier],
        'Label_File': labels[identifier]
    })



 set up a data loader for my structured data, considering the additional elevation channel

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [17]:
# class MultiChannelDataGenerator(Sequence):
#     def __init__(self, image_paths, elevation_paths, labels_paths, batch_size=32, dim=(128, 128), n_channels=5, n_classes=11, shuffle=True):
#         'Initialization'
#         self.dim = dim
#         self.batch_size = batch_size
#         self.image_paths = image_paths
#         self.elevation_paths = elevation_paths
#         self.labels_paths = labels_paths
#         self.n_channels = n_channels
#         self.n_classes = n_classes
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.image_paths) / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_image_temp = [self.image_paths[k] for k in indexes]
#         list_elevation_temp = [self.elevation_paths[k] for k in indexes]
#         list_labels_temp = [self.labels_paths[k] for k in indexes]

#         # Generate data
#         X, y = self.__data_generation(list_image_temp, list_elevation_temp, list_labels_temp)

#         return X, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.image_paths))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_image_temp, list_elevation_temp, list_labels_temp):
#         'Generates data containing batch_size samples'
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)

#         # Generate data
#         for i, (img_path, elev_path, label_path) in enumerate(zip(list_image_temp, list_elevation_temp, list_labels_temp)):
#             # Store sample
#             img = load_img(img_path, color_mode='rgba', target_size=self.dim)
#             img = img_to_array(img)[:,:,:4]  # Ensure img has only 4 channels (RGBA or RGBNIR)

#             elevation = load_img(elev_path, color_mode='grayscale', target_size=self.dim)
#             elevation = img_to_array(elevation)
#             elevation = np.repeat(elevation, 4, axis=-1)  # Repeat elevation across what would be the color channels to match dimensions

#             X[i,] = np.concatenate((img, elevation), axis=-1)

#             # Store class
#             label = load_img(label_path, color_mode='grayscale', target_size=self.dim)
#             label = img_to_array(label)
#             y[i] = tf.keras.utils.to_categorical(label, num_classes=self.n_classes)

#         return X, np.argmax(y, axis=-1)


end

In [18]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        image_batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        label_batch_files = self.label_files[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images, batch_labels = self.load_images_and_labels(image_batch_files,label_batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)

            return image

    def load_images_and_labels(self, image_files, label_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            # image[image <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(image)):
            #     nan_mask = np.isnan(image)
            #     image[nan_mask] = np.nanmean(image)

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(image)):
            #     inf_mask = np.isinf(image)
            #     image[inf_mask] = np.nanmean(image)

            # # Convert to float
            # image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            images.append(image)

        for label_file in label_files:
            # label_file = image_file.replace("planet", "ccap_filled").replace('images_2018','ccap_filled')
            label = self.load_and_reshape_image(label_file)
            # label[label <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(label)):
            #     nan_mask = np.isnan(label)
            #     label[nan_mask] = 12

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(label)):
            #     inf_mask = np.isinf(label)
            #     label[inf_mask] = 12

            label -= 1
            label = label.astype(np.int8)
            label = to_categorical(label, num_classes=self.num_classes)
            labels.append(label)

        return np.array(images), np.array(labels)

In [20]:
# print(train_label_paths[0])
# train_image_paths[0]

In [21]:
# /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/images_2018

In [22]:
# data_gen_test = DataGenerator(sum_df['Images_path'][0:5], sum_df['Labels_path'][0:5], 512, 512, 2, 22)
# Create the data generator
training_data_generator = DataGenerator(train_image_paths, train_label_paths, 128, 128, 13, 12)
# validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 25)


In [23]:
# /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ccap_filled/1004_ccap_filled.tif

In [24]:
# batch_images, batch_labels = training_data_generator.__getitem__(0)

In [25]:
# def create_model(img_size,num_bands, num_classes):
#     inputs = keras.Input(shape=(img_size[0], img_size[1], num_bands))  # Change the number of channels to 4

#     # Entry block
#     x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("LeakyReLU")(x)

#     previous_block_activation = x  # Set aside residual

#     # Blocks 1, 2, 3 are identical apart from the feature depth.
#     for filters in [64, 128, 256]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#         # Project residual
#         residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
#             previous_block_activation
#         )
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     for filters in [256, 128, 64, 32]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.UpSampling2D(2)(x)

#         residual = layers.UpSampling2D(2)(previous_block_activation)
#         residual = layers.Conv2D(filters, 1, padding="same")(residual)
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     # Add a per-pixel classification layer
#     outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

#     # Define the model
#     model = keras.Model(inputs, outputs)
#     model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # updated to sparse categorical cross-entropy loss
#     return model

In [26]:

# from tensorflow.python.keras.layers.normalization import BatchNormalization
# from tensorflow.python.keras.layers.normalization import BatchNormalization

# from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization.batch_normalization import BatchNormalization
# from tensorflow.python.keras.layers import BatchNormalization

# U-Net model for image segmentation.
# Encoder and decoder conncted by a center block.
# Encoder downsamples the input image while capturing its features.
# Decoder upsamples the encoded image to generate a segmentation map.

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(CLASSES, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [27]:
KERNEL_SIZE = 128
CLASSES = 12
BANDS = range(13)
KERNEL_SHAPE = [128, 128]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [28]:
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [29]:
# model = create_model([128,128],4,11)
model = get_model()

In [30]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 13)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         3776      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 32)         0         ['batch_normalization[0][0

In [31]:
# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_accuracy'])
callbacks = [
    keras.callbacks.ModelCheckpoint("landcover_segmentation.h5", save_best_only=True)
]

model.fit(training_data_generator, epochs=10, callbacks=callbacks,shuffle=True)

Epoch 1/10
95/95 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241

95/95 [==============================] - 238s 2s/step - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241
Epoch 2/10
95/95 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241

95/95 [==============================] - 184s 2s/step - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241
Epoch 3/10
95/95 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241

95/95 [==============================] - 190s 2s/step - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241
Epoch 4/10
95/95 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241

95/95 [==============================] - 177s 2s/step - loss: nan - accuracy: 0.2241 - categorical_accuracy: 0.2241
Epoch 5/10
62/95 [==================>...........] - ETA: 1:02 - loss: nan - accuracy: 0.2074 - categorical_accuracy: 0.2074

KeyboardInterrupt: 